# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import PicklingError
from joblib import dump, load
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, recall_score, classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, matthews_corrcoef, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from datetime import datetime
from tensorflow import keras
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.utils.data import evaluate_print
from IPython.display import display, Markdown
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Settings
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "device_single_user"
TIME_BUCKETS  = [
    "7d", 
    "2d", 
    "1d", 
    "12h",
    "6h", 
    "1h"
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def add_count_and_threat(x):
    threat = 1 if 1 in x.threat.values else 0
    return pd.Series([len(x), threat], index=["count", "threat"])


def train_classifier_on_single_users(df, classifier, time_bucket, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in df.groupby("user"):

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])


            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group[["count"]])
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group[["count"]])
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, time_bucket, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.time_bucket = time_bucket
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=1
                    )
            except (ValueError, IndexError) as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    """Function that orchestrates the training process.

    Args:
        df (pandas.DataFrame): The dataframe with the training data.
        classifier (sklearn.base.BaseEstimator): The pyod classifier that
            implements the interfaces from the sklearn BaseEstimator class.
        parameters (dict): The parameters that should be tested as part of
            the grid search.

    Returns:
        bool: The return value. True for success, False otherwise.

    """
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, time_bucket in enumerate(TIME_BUCKETS, start=1):

        display(Markdown(f"# {i}/{len(TIME_BUCKETS)} - Bucket: {time_bucket}"))

        # Group the dataframe by users and time buckets, fill missing dates with 0
        tmp = df.groupby([df["user"], pd.Grouper(freq=time_bucket, key="date")]).apply(add_count_and_threat)
        tmp = tmp.reindex(pd.MultiIndex.from_product(tmp.index.levels))
        tmp = tmp.fillna(0)

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=tmp,
            classifier=classifier,
            time_bucket=time_bucket,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "device"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 24 different hyperparameter combinations
 Training Time: 3.4469    Inference Time: 1.0439    pAUC: 0.7495 ± 0.0088   Recall: 0.1721 ± 0.0227   TN: 18756.0   FP: 145.0   FN: 147.0   TP: 32.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 32.1186   Inference Time: 2.1501    pAUC: 0.7841 ± 0.0022   Recall: 0.2324 ± 0.0115   TN: 18749.0   FP: 152.0   FN: 141.0   TP: 38.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 40.9587   Inference Time: 6.5077    pAUC: 0.7869 ± 0.0013   Recall: 0.2413 ± 0.0042   TN: 18754.0   FP: 147.0   FN: 136.0   TP: 43.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 78.2864   Inference Time: 11.7285   pAUC: 0.7869 ± 0.0008   Recall: 0.2436 ± 0.0045   TN: 18754.0   FP: 147.0   FN: 136.0   TP: 43.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 3.4569    Inference Time: 1.0566    pAUC: 0.7495 ± 0.0088   Recall: 0.1721 ± 0.0227   TN: 18756.0   FP: 145.0   FN: 147.0   TP: 32.0   

# 2/6 - Bucket: 2d

Testing 24 different hyperparameter combinations
 Training Time: 3.5501    Inference Time: 1.0980    pAUC: 0.7351 ± 0.0106   Recall: 0.2154 ± 0.0212   TN: 65737.0   FP: 253.0   FN: 207.0   TP: 53.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 32.6304   Inference Time: 2.3917    pAUC: 0.8071 ± 0.0029   Recall: 0.3577 ± 0.0049   TN: 65620.0   FP: 370.0   FN: 166.0   TP: 94.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 43.5261   Inference Time: 7.4382    pAUC: 0.8103 ± 0.0009   Recall: 0.3600 ± 0.0019   TN: 65621.0   FP: 369.0   FN: 166.0   TP: 94.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 82.2172   Inference Time: 13.6852   pAUC: 0.8106 ± 0.0004   Recall: 0.3615 ± 0.0000   TN: 65617.0   FP: 373.0   FN: 166.0   TP: 94.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 3.5129    Inference Time: 1.0782    pAUC: 0.7837 ± 0.0026   Recall: 0.3123 ± 0.0176   TN: 65583.0   FP: 407.0   FN: 173.0   TP: 87.0   

# 3/6 - Bucket: 1d

...

Testing 24 different hyperparameter combinations
 Training Time: 3.5843    Inference Time: 1.1445    pAUC: 0.6258 ± 0.0058   Recall: 0.1791 ± 0.0131   TN: 263987.0   FP: 535.0   FN: 382.0   TP: 96.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 33.0283   Inference Time: 2.6250    pAUC: 0.7341 ± 0.0023   Recall: 0.4586 ± 0.0043   TN: 263480.0   FP: 1042.0   FN: 256.0   TP: 222.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 51.2263   Inference Time: 8.8967    pAUC: 0.7403 ± 0.0004   Recall: 0.4707 ± 0.0000   TN: 263456.0   FP: 1066.0   FN: 253.0   TP: 225.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 92.0916   Inference Time: 17.1855   pAUC: 0.7405 ± 0.0001   Recall: 0.4707 ± 0.0000   TN: 263450.0   FP: 1072.0   FN: 253.0   TP: 225.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 3.7459    Inference Time: 1.2051    pAUC: 0.6672 ± 0.0171   Recall: 0.3071 ± 0.0353   TN: 263629.0   FP: 893.0   FN: 302.0   

# 5/6 - Bucket: 6h

Testing 24 different hyperparameter combinations
 Training Time: 3.8938    Inference Time: 1.2610    pAUC: 0.6131 ± 0.0128   Recall: 0.1394 ± 0.0311   TN: 528064.0   FP: 1130.0   FN: 451.0   TP: 90.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 34.0864   Inference Time: 3.1378    pAUC: 0.7273 ± 0.0074   Recall: 0.4322 ± 0.0188   TN: 526908.0   FP: 2286.0   FN: 293.0   TP: 248.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 56.1094   Inference Time: 10.6691   pAUC: 0.7441 ± 0.0006   Recall: 0.4787 ± 0.0000   TN: 526817.0   FP: 2377.0   FN: 282.0   TP: 259.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 101.3080   Inference Time: 20.4507   pAUC: 0.7446 ± 0.0006   Recall: 0.4787 ± 0.0000   TN: 526798.0   FP: 2396.0   FN: 282.0   TP: 259.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 4.1084    Inference Time: 1.3478    pAUC: 0.6472 ± 0.0161   Recall: 0.2392 ± 0.0377   TN: 527352.0   FP: 1842.0   FN: 393.0

# 6/6 - Bucket: 1h

Testing 24 different hyperparameter combinations
 Training Time: 5.3407    Inference Time: 1.9080    pAUC: 0.5788 ± 0.0092   Recall: 0.0604 ± 0.0137   TN: 3172632.0   FP: 4159.0   FN: 1280.0   TP: 74.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 40.9342   Inference Time: 6.4423    pAUC: 0.6522 ± 0.0023   Recall: 0.2245 ± 0.0058   TN: 3164565.0   FP: 12226.0   FN: 1042.0   TP: 312.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 105.8197   Inference Time: 25.5214   pAUC: 0.6704 ± 0.0010   Recall: 0.2780 ± 0.0019   TN: 3163885.0   FP: 12906.0   FN: 976.0   TP: 378.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 180.2202   Inference Time: 48.9384   pAUC: 0.6726 ± 0.0002   Recall: 0.2840 ± 0.0006   TN: 3163868.0   FP: 12923.0   FN: 969.0   TP: 385.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 5.3873    Inference Time: 1.9193    pAUC: 0.6067 ± 0.0072   Recall: 0.1115 ± 0.0138   TN: 3169026.0   FP: 7765.0 

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 28 different hyperparameter combinations
 Training Time: 4.6368    Inference Time: 1.2676    pAUC: 0.7107 ± 0.0000   Recall: 0.0670 ± 0.0000   TN: 18723.0   FP: 178.0   FN: 167.0   TP: 12.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 6.1265    Inference Time: 1.3767    pAUC: 0.7095 ± 0.0000   Recall: 0.0503 ± 0.0000   TN: 18723.0   FP: 178.0   FN: 170.0   TP: 9.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 7.7868    Inference Time: 1.5049    pAUC: 0.7014 ± 0.0000   Recall: 0.0782 ± 0.0000   TN: 18727.0   FP: 174.0   FN: 165.0   TP: 14.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 9.6177    Inference Time: 1.6330    pAUC: 0.7093 ± 0.0000   Recall: 0.0559 ± 0.0000   TN: 18725.0   FP: 176.0   FN: 169.0   TP: 10.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 4.4888    Inference Time: 1.2404    pAUC: 0.5517 ± 0.0000   Recall: 0.0615 ± 0.0000   TN: 18719.0   FP: 182.0   FN: 168.0   TP: 11.0   Params: {'n_bins': 8, 'n

# 2/6 - Bucket: 2d

Testing 28 different hyperparameter combinations
 Training Time: 3.5089    Inference Time: 0.9739    pAUC: 0.5775 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65379.0   FP: 611.0   FN: 259.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.8551    Inference Time: 1.1775    pAUC: 0.5927 ± 0.0000   Recall: 0.0115 ± 0.0000   TN: 65351.0   FP: 639.0   FN: 257.0   TP: 3.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.9626    Inference Time: 1.4960    pAUC: 0.5864 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65285.0   FP: 705.0   FN: 259.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 7.4977    Inference Time: 1.4835    pAUC: 0.5879 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65373.0   FP: 617.0   FN: 259.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 3.4644    Inference Time: 0.9457    pAUC: 0.5478 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65990.0   FP: 0.0     FN: 260.0   TP: 0.0   Params: {'n_bins': 8, 'n_ran

# 3/6 - Bucket: 1d

...

Testing 28 different hyperparameter combinations
 Training Time: 4.1349    Inference Time: 1.3298    pAUC: 0.7548 ± 0.0000   Recall: 0.2050 ± 0.0000   TN: 261958   FP: 2564    FN: 380   TP: 98    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 5.8821    Inference Time: 1.6770    pAUC: 0.7564 ± 0.0000   Recall: 0.2322 ± 0.0000   TN: 261987   FP: 2535    FN: 367   TP: 111   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 7.8532    Inference Time: 2.0581    pAUC: 0.7551 ± 0.0000   Recall: 0.2280 ± 0.0000   TN: 261929   FP: 2593    FN: 369   TP: 109   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 9.3315    Inference Time: 2.3241    pAUC: 0.7671 ± 0.0000   Recall: 0.2301 ± 0.0000   TN: 261977   FP: 2545    FN: 368   TP: 110   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 4.2094    Inference Time: 1.3539    pAUC: 0.6421 ± 0.0000   Recall: 0.0481 ± 0.0000   TN: 261701   FP: 2821    FN: 455   TP: 23    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Bucket: 6h

Testing 28 different hyperparameter combinations
 Training Time: 4.4757    Inference Time: 1.5301    pAUC: 0.8050 ± 0.0000   Recall: 0.3216 ± 0.0000   TN: 524099   FP: 5095    FN: 367   TP: 174   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 6.6643    Inference Time: 2.1319    pAUC: 0.7994 ± 0.0000   Recall: 0.3050 ± 0.0000   TN: 524084   FP: 5110    FN: 376   TP: 165   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 8.8067    Inference Time: 2.7141    pAUC: 0.8062 ± 0.0000   Recall: 0.3420 ± 0.0000   TN: 524005   FP: 5189    FN: 356   TP: 185   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 11.5655   Inference Time: 3.4109    pAUC: 0.8145 ± 0.0000   Recall: 0.3660 ± 0.0000   TN: 524016   FP: 5178    FN: 343   TP: 198   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 4.4386    Inference Time: 1.5297    pAUC: 0.7625 ± 0.0000   Recall: 0.0739 ± 0.0000   TN: 524131   FP: 5063    FN: 501   TP: 40    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Bucket: 1h

Testing 28 different hyperparameter combinations
 Training Time: 11.6462   Inference Time: 5.6221    pAUC: 0.8833 ± 0.0000   Recall: 0.2430 ± 0.0000   TN: 3145244   FP: 31547   FN: 1025   TP: 329   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 20.5718   Inference Time: 10.0611   pAUC: 0.8948 ± 0.0000   Recall: 0.2555 ± 0.0000   TN: 3145308   FP: 31483   FN: 1008   TP: 346   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 30.3039   Inference Time: 14.5493   pAUC: 0.9102 ± 0.0000   Recall: 0.2969 ± 0.0000   TN: 3145650   FP: 31141   FN: 952   TP: 402   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 38.6685   Inference Time: 18.5893   pAUC: 0.9019 ± 0.0000   Recall: 0.2888 ± 0.0000   TN: 3145198   FP: 31593   FN: 963   TP: 391   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 12.0947   Inference Time: 5.9440    pAUC: 0.8992 ± 0.0000   Recall: 0.2962 ± 0.0000   TN: 3145389   FP: 31402   FN: 953   TP: 401   Params: {'n_bins': 8, 'n_random_cuts

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 2.5021    Inference Time: 1.0708    pAUC: 0.7819 ± 0.0000   Recall: 0.2402 ± 0.0000   TN: 18737    FP: 164     FN: 136   TP: 43    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7819 ± 0.0000
Recall: 0.2402 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 2.6815    Inference Time: 1.1687    pAUC: 0.9351 ± 0.0000   Recall: 0.6115 ± 0.0000   TN: 65533    FP: 457     FN: 101   TP: 159   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9351 ± 0.0000
Recall: 0.6115 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 2.5689    Inference Time: 1.2000    pAUC: 0.8865 ± 0.0000   Recall: 0.5314 ± 0.0000   TN: 261921   FP: 2601    FN: 224   TP: 254   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8865 ± 0.0000
Recall: 0.5314 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 2.7511    Inference Time: 1.3759    pAUC: 0.8973 ± 0.0000   Recall: 0.6488 ± 0.0000   TN: 524271   FP: 4923    FN: 190   TP: 351   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8973 ± 0.0000
Recall: 0.6488 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 3.4330    Inference Time: 2.5618    pAUC: 0.9284 ± 0.0000   Recall: 0.4114 ± 0.0000   TN: 3145669   FP: 31122   FN: 797   TP: 557   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9284 ± 0.0000
Recall: 0.4114 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 2.7902    Inference Time: 1.2065    pAUC: 0.7905 ± 0.0000   Recall: 0.2849 ± 0.0000   TN: 18656    FP: 245     FN: 128   TP: 51    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7905 ± 0.0000
Recall: 0.2849 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 3.1434    Inference Time: 1.3494    pAUC: 0.9237 ± 0.0000   Recall: 0.6115 ± 0.0000   TN: 65426    FP: 564     FN: 101   TP: 159   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9237 ± 0.0000
Recall: 0.6115 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 2.9042    Inference Time: 1.3901    pAUC: 0.8841 ± 0.0000   Recall: 0.5230 ± 0.0000   TN: 262051   FP: 2471    FN: 228   TP: 250   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8841 ± 0.0000
Recall: 0.5230 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 2.8194    Inference Time: 1.4112    pAUC: 0.8973 ± 0.0000   Recall: 0.6488 ± 0.0000   TN: 524271   FP: 4923    FN: 190   TP: 351   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8973 ± 0.0000
Recall: 0.6488 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 3.4247    Inference Time: 2.5644    pAUC: 0.9284 ± 0.0000   Recall: 0.4114 ± 0.0000   TN: 3145669   FP: 31122   FN: 797   TP: 557   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9284 ± 0.0000
Recall: 0.4114 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [3, 5, 6],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 6/1 - Bucket: 1h

Testing 120 different hyperparameter combinations
 Training Time: 13.8892   Inference Time: 1.7213    pAUC: 0.5990 ± 0.0012   Recall: 0.0136 ± 0.0004   TN: 3145109.0   FP: 31682.0   FN: 1336.0   TP: 18.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 3, 'use_weights': True}
 Training Time: 13.6125   Inference Time: 1.7036    pAUC: 0.8656 ± 0.0005   Recall: 0.2112 ± 0.0000   TN: 3145069.0   FP: 31722.0   FN: 1068.0   TP: 286.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 3, 'use_weights': False}
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 5, 'use_weights': True}: index 3 is out of bounds for axis 0 with size 3
 Training Time: 18.4457   Inference Time: 1.6992    pAUC: 0.8503 ± 0.0012   Recall: 0.2018 ± 0.0116   TN: 3145300.0   FP: 31491.0   FN: 1068.0   TP: 286.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 5, 'use_weights': False}
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 6, 'use_weights': True}: index 5 is out of bounds for axis 0 with size 5
 Training Time: 21

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 168 different hyperparameter combinations
 Training Time: 2.4057    Inference Time: 0.8089    pAUC: 0.6045 ± 0.0000   Recall: 0.1955 ± 0.0000   TN: 18747    FP: 154     FN: 144   TP: 35    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 2.6619    Inference Time: 0.9039    pAUC: 0.6045 ± 0.0000   Recall: 0.1955 ± 0.0000   TN: 18747    FP: 154     FN: 144   TP: 35    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 2.4349    Inference Time: 0.8155    pAUC: 0.6045 ± 0.0000   Recall: 0.1955 ± 0.0000   TN: 18747    FP: 154     FN: 144   TP: 35    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.4877    Inference Time: 0.8393    pAUC: 0.6045 ± 0.0000   Recall: 0.1955 ± 0.0000   TN: 18747    FP: 154     FN: 144   TP: 35    Params: {'n_components': 1, 'stan

# 2/6 - Bucket: 2d

Testing 168 different hyperparameter combinations
 Training Time: 2.4002    Inference Time: 0.8016    pAUC: 0.6575 ± 0.0000   Recall: 0.2654 ± 0.0000   TN: 65398    FP: 592     FN: 191   TP: 69    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 2.5182    Inference Time: 0.8436    pAUC: 0.6575 ± 0.0000   Recall: 0.2654 ± 0.0000   TN: 65398    FP: 592     FN: 191   TP: 69    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 2.6221    Inference Time: 0.8876    pAUC: 0.6575 ± 0.0000   Recall: 0.2654 ± 0.0000   TN: 65398    FP: 592     FN: 191   TP: 69    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.5368    Inference Time: 0.8506    pAUC: 0.6575 ± 0.0000   Recall: 0.2654 ± 0.0000   TN: 65398    FP: 592     FN: 191   TP: 69    Params: {'n_components': 1, 'stan

# 3/6 - Bucket: 1d

...

Testing 168 different hyperparameter combinations
 Training Time: 2.3940    Inference Time: 0.8180    pAUC: 0.7461 ± 0.0000   Recall: 0.4331 ± 0.0000   TN: 261869   FP: 2653    FN: 271   TP: 207   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 2.3778    Inference Time: 0.8062    pAUC: 0.7461 ± 0.0000   Recall: 0.4331 ± 0.0000   TN: 261869   FP: 2653    FN: 271   TP: 207   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 2.4426    Inference Time: 0.8370    pAUC: 0.7461 ± 0.0000   Recall: 0.4331 ± 0.0000   TN: 261869   FP: 2653    FN: 271   TP: 207   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.4084    Inference Time: 0.8204    pAUC: 0.7461 ± 0.0000   Recall: 0.4331 ± 0.0000   TN: 261869   FP: 2653    FN: 271   TP: 207   Params: {'n_components': 1, 'stan

# 5/6 - Bucket: 6h

Testing 168 different hyperparameter combinations
 Training Time: 3.1412    Inference Time: 1.1085    pAUC: 0.8478 ± 0.0000   Recall: 0.5915 ± 0.0000   TN: 524189   FP: 5005    FN: 221   TP: 320   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 3.0169    Inference Time: 1.0314    pAUC: 0.8478 ± 0.0000   Recall: 0.5915 ± 0.0000   TN: 524189   FP: 5005    FN: 221   TP: 320   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 3.0538    Inference Time: 1.0531    pAUC: 0.8478 ± 0.0000   Recall: 0.5915 ± 0.0000   TN: 524189   FP: 5005    FN: 221   TP: 320   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.9795    Inference Time: 1.0223    pAUC: 0.8478 ± 0.0000   Recall: 0.5915 ± 0.0000   TN: 524189   FP: 5005    FN: 221   TP: 320   Params: {'n_components': 1, 'stan

# 6/6 - Bucket: 1h

Testing 168 different hyperparameter combinations
 Training Time: 3.3308    Inference Time: 1.2250    pAUC: 0.8666 ± 0.0000   Recall: 0.5399 ± 0.0000   TN: 3145931   FP: 30860   FN: 623   TP: 731   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 3.3344    Inference Time: 1.2288    pAUC: 0.8666 ± 0.0000   Recall: 0.5399 ± 0.0000   TN: 3145931   FP: 30860   FN: 623   TP: 731   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 3.3073    Inference Time: 1.2066    pAUC: 0.8666 ± 0.0000   Recall: 0.5399 ± 0.0000   TN: 3145931   FP: 30860   FN: 623   TP: 731   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 3.3724    Inference Time: 1.2414    pAUC: 0.8666 ± 0.0000   Recall: 0.5399 ± 0.0000   TN: 3145931   FP: 30860   FN: 623   TP: 731   Params: {'n_components': 1, '

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 969.6489   Inference Time: 16.2065   pAUC: 0.6386 ± 0.0000   Recall: 0.1564 ± 0.0000   TN: 18738    FP: 163     FN: 151   TP: 28    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1049.0304   Inference Time: 16.3444   pAUC: 0.6337 ± 0.0000   Recall: 0.1564 ± 0.0000   TN: 18738    FP: 163     FN: 151   TP: 28    Params: {'hidden_activation': 'sigmoid'

# 2/6 - Bucket: 2d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1155.2358   Inference Time: 19.7633   pAUC: 0.7079 ± 0.0000   Recall: 0.1577 ± 0.0000   TN: 65376    FP: 614     FN: 219   TP: 41    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1234.0749   Inference Time: 19.5287   pAUC: 0.6894 ± 0.0000   Recall: 0.1423 ± 0.0000   TN: 65365    FP: 625     FN: 223   TP: 37    Params: {'hidden_activation': 'sigmoid

# 3/6 - Bucket: 1d

...

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1192.1607   Inference Time: 26.2647   pAUC: 0.8753 ± 0.0000   Recall: 0.4519 ± 0.0000   TN: 262153   FP: 2369    FN: 262   TP: 216   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1170.0570   Inference Time: 26.8862   pAUC: 0.8692 ± 0.0000   Recall: 0.4561 ± 0.0000   TN: 262094   FP: 2428    FN: 260   TP: 218   Params: {'hidden_activation': 'sigmoid

# 5/6 - Bucket: 6h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1218.6070   Inference Time: 36.4300   pAUC: 0.8744 ± 0.0000   Recall: 0.5545 ± 0.0000   TN: 524386   FP: 4808    FN: 241   TP: 300   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1209.8343   Inference Time: 41.7662   pAUC: 0.8646 ± 0.0000   Recall: 0.5120 ± 0.0000   TN: 524206   FP: 4988    FN: 264   TP: 277   Params: {'hidden_activation': 'sigmoid

# 6/6 - Bucket: 1h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1350.1663   Inference Time: 150.0944   pAUC: 0.8452 ± 0.0000   Recall: 0.1861 ± 0.0000   TN: 3145484   FP: 31307   FN: 1102   TP: 252   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 1217.9350   Inference Time: 146.4752   pAUC: 0.8580 ± 0.0000   Recall: 0.1987 ± 0.0000   TN: 3145393   FP: 31398   FN: 1085   TP: 269   Params: {'hidden_activation': 's

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)